In [5]:
from __future__ import annotations

from typing import Final

import numpy as np
import pandas as pd

# Passive excess returns

In [34]:
# STATS

# correlations
R_CORR: Final[pd.DataFrame] = pd.DataFrame(
    {
        "cash": {
            "cash": 1.00,
            "dm": 0.00,
            "em": 0.00,
            "distressed": 0.00,
        },
        "dm": {
            "cash": 0.00,
            "dm": 1.00,
            "em": 0.80,
            "distressed": 0.50,
        },
        "em": {
            "cash": 0.00,
            "dm": 0.80,
            "em": 1.00,
            "distressed": 0.50,
        },
        "distressed": {
            "cash": 0.00,
            "dm": 0.50,
            "em": 0.50,
            "distressed": 1.00,
        },
    }
)

# volatilities
R_VOL: Final[pd.Series] = pd.Series(
    {
        "cash": 0.00,
        "dm": 0.15,
        "em": 0.30,
        "distressed": 0.60,
    }
)

# sharpes
R_SHARPE: Final[pd.Series] = pd.Series(
    {
        # really NaN (0/0), but this makes the calcs work
        "cash": 0.00,
        # neglect BAB
        "dm": 0.50,
        "em": 0.50,
        "distressed": 0.50,
    }
)

# expected returns
R_ER: Final[pd.Series] = R_VOL * R_SHARPE

In [35]:
# UTILS

def _get_r_predint(
    r_vol: float = 0.10,
    r_sharpe: float = 1.00,
    crit: float = 1.96,
) -> pd.Series:
    """Get 95% return-prediction interval, based on return stats."""
    r_er = r_vol * r_sharpe
    radius = r_vol * crit
    return pd.Series(
        {
            "lower": r_er - radius,
            "upper": r_er + radius,
        }
    )

# Portfolio

In [36]:
# STATS

# initial positions on NAV, from most to least liquid
PFLIO_W: Final[pd.Series] = pd.Series(
    {
        "cash": 0.10,
        "dm": 0.30,
        "em": 0.30,
        "distressed": 0.30,
    }
)

In [37]:
# UTILS

def _check_pflio_w(
    pflio_w: pd.Series = PFLIO_W,
) -> pd.Series:
    """We include cash, so this should tie out."""
    nlev = pflio_w.sum()
    is_bad = not np.isclose(nlev, 1.00)
    if is_bad:
        raise ValueError(pflio_w)
    return pflio_w

def __get_pflio_vol(
    r_corr: pd.DataFrame = R_CORR,
    r_vol: pd.Series = R_VOL,
    pflio_w: pd.Series = PFLIO_W,
) -> float:
    pflio_vw = r_vol * pflio_w
    pflio_var = pflio_vw @ r_corr @ pflio_vw
    return pflio_var**0.5

def __get_pflio_er(
    r_er: pd.Series = R_ER,
    pflio_w: pd.Series = PFLIO_W,
) -> float:
    return r_er @ pflio_w

def __get_pflio_sharpe(
    r_corr: pd.DataFrame = R_CORR,
    r_vol: pd.Series = R_VOL,
    r_er: pd.Series = R_ER,
    pflio_w: pd.Series = PFLIO_W,
) -> float:
    pflio_vol = __get_pflio_vol(
        r_corr=r_corr,
        r_vol=r_vol,
        pflio_w=pflio_w,
    )
    pflio_er = __get_pflio_er(
        r_er=r_er,
        pflio_w=pflio_w,
    )
    return pflio_er / pflio_vol

def _get_pflio_stats(
    r_corr: pd.DataFrame = R_CORR,
    r_vol: pd.Series = R_VOL,
    r_er: pd.Series = R_ER,
    pflio_w: pd.Series = PFLIO_W,
) -> pd.Series:
    return pd.Series(
        {
            "vol": __get_pflio_vol(
                r_corr=r_corr,
                r_vol=r_vol,
                pflio_w=pflio_w,
            ),
            "sharpe": __get_pflio_sharpe(
                r_corr=r_corr,
                r_vol=r_vol,
                r_er=r_er,
                pflio_w=pflio_w,
            ),
        }
    )

def _get_pflio_predint(
    r_corr: pd.DataFrame = R_CORR,
    r_vol: pd.Series = R_VOL,
    r_er: pd.Series = R_ER,
    pflio_w: pd.Series = PFLIO_W,
) -> pd.Series:
    pflio_stats = _get_pflio_stats(
        r_corr=r_corr,
        r_vol=r_vol,
        r_er=r_er,
        pflio_w=pflio_w,
    )
    return _get_r_predint(
        r_vol=pflio_stats["vol"],
        r_sharpe=pflio_stats["sharpe"],
    )

In [1]:
_check_pflio_w()

In [38]:
_get_pflio_stats()

vol       0.270749
sharpe    0.581720
dtype: float64

In [39]:
_get_pflio_predint()

lower   -0.373168
upper    0.688168
dtype: float64

# Subscription-Redemption-Transfer scenarios

In [69]:
def run(
    title: str = "PROD",
    pflio_w_init: pd.Series = PFLIO_W,
    # note: can simulate drawing on line of credit by redeeming more cash than on-hand
    pflio_srt: pd.Series = pd.Series(0, index=PFLIO_W.index),
) -> None:
    pflio_glev_init = pflio_w_init.abs().sum()
    pflio_nlev_init = pflio_w_init.sum()
    pflio_srt_amt = pflio_srt.sum()
    pflio_nav_final = 1 + pflio_srt_amt
    pflio_w_final = (pflio_w_init + pflio_srt) / pflio_nav_final
    pflio_w_final = _check_pflio_w(pflio_w=pflio_w_final)
    pflio_predint_final = _get_pflio_predint(pflio_w=pflio_w_final)
    print(f"Scenario {title}:")
    print(f"Portfolio is initially levered to {pflio_glev_init :.2f}x gross and {pflio_nlev_init :.2f}x net.")
    print(f"SRT totals {pflio_srt_amt *100 :.0f}% of NAV.")
    print(f"""Without rebalancing, post-SRT forecast return over the next quarter is [{pflio_predint_final["lower"]/4 *100:.1f}, {pflio_predint_final["upper"]/4 *100:.1f}]%.""")

In [70]:
run()

Scenario PROD:
Portfolio is initially levered to 1.00x gross and 1.00x net.
Redemption totals 0% of NAV.
Without rebalancing, post-redemption forecast return over the next quarter is [-9.3, 17.2]%.
